# 📊 Preparador de Lora de Hollowstrawberry

Basado en el trabajo de [Kohya_ss](https://github.com/kohya-ss/sd-scripts) y [Linaqruf](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb#scrollTo=-Z4w3lfFKLjr). ¡Gracias!

### ⭕ Disclaimer
The purpose of this document is to research bleeding-edge technologies in the field of machine learning.  
Please read and follow the [Google Colab guidelines](https://research.google.com/colaboratory/faq.html) and its [Terms of Service](https://research.google.com/colaboratory/tos_v3.html).

| |GitHub|🇬🇧 English|🇪🇸 Spanish|
|:--|:-:|:-:|:-:|
| 🏠 **Origen** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab) | | |
| 📊 **Dataset Maker** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Dataset_Maker.ipynb) |
| ⭐ **Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) |
| 🌟 **XL Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) |  |
| 🌟 **Legacy XL Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL_Legacy.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL_Legacy.ipynb) |  |

In [ ]:
import os
from IPython import get_ipython
from IPython.display import display, Markdown

COLAB = True

if COLAB:
  from google.colab.output import clear as clear_output
else:
  from IPython.display import clear_output

#@title ## 🚩 Empezar aquí

#@markdown ### 1️⃣  Inicio
#@markdown Esta celda cargará algunos requerimientos y creará las carpetas correspondientes en tu Google Drive. <p>
#@markdown Tu nombre de proyecto será la carpeta donde trabajaremos. No se permiten espacios.
nombre_proyecto = "" #@param {type:"string"}
project_name = nombre_proyecto.strip()
#@markdown La estructura de carpetas no importa y es por comodidad. Asegúrate de siempre elegir la misma. Me gusta organizar por proyecto.
estructura_de_carpetas = "Organizar por proyecto (MyDrive/Loras/nombre_proyecto/dataset)" #@param ["Organizar por categoría (MyDrive/lora_training/datasets/nombre_proyecto)", "Organizar por proyecto (MyDrive/Loras/nombre_proyecto/dataset)"]
folder_structure = estructura_de_carpetas

if not project_name or any(c in project_name for c in " .()\"'\\") or project_name.count("/") > 1:
  print("Por favor elige un nombre válido.")
else:
  if COLAB and not os.path.exists('/content/drive'):
    from google.colab import drive
    print("📂 Conectando a Google Drive...")
    drive.mount('/content/drive')

  project_base = project_name if "/" not in project_name else project_name[:project_name.rfind("/")]
  project_subfolder = project_name if "/" not in project_name else project_name[project_name.rfind("/")+1:]

  root_dir = "/content" if COLAB else "~/Loras"
  deps_dir = os.path.join(root_dir, "deps")

  if "/Loras" in folder_structure:
    main_dir      = os.path.join(root_dir, "drive/MyDrive/Loras") if COLAB else root_dir
    config_folder = os.path.join(main_dir, project_base)
    images_folder = os.path.join(main_dir, project_base, "dataset")
    if "/" in project_name:
      images_folder = os.path.join(images_folder, project_subfolder)
  else:
    main_dir      = os.path.join(root_dir, "drive/MyDrive/lora_training") if COLAB else root_dir
    config_folder = os.path.join(main_dir, "config", project_name)
    images_folder = os.path.join(main_dir, "datasets", project_name)

  for dir in [main_dir, deps_dir, images_folder, config_folder]:
    os.makedirs(dir, exist_ok=True)

  print(f"✅ ¡Proyecto {project_name} listo!")
  step1_installed_flag = True


In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Por favor usa el paso 1 primero.")

import json
import time
from urllib.request import urlopen, Request

#@markdown ### 2️⃣ Obtener imágenes

#@markdown Obtendremos imágenes de la galería de anime llamada [Gelbooru](https://gelbooru.com/). Las imágenes se organizan por miles de tags que describen todo acerca de una imagen. <p>
#@markdown * Si quieres encontrar y usar tus propias imágenes, ponlas dentro de la carpeta `Loras/nombre_proyecto/dataset` en tu Google Drive.
#@markdown * Si quieres descargar capturas de episodios de anime, existe [este otro colab de otra persona](https://colab.research.google.com/drive/1oBSntB40BKzNmKceXUlkXzujzdQw-Ci7) aunque aquel es más complicado.

#@markdown Hasta 1000 imágenes se descargarán en un minuto, no debes abusar de ello. <p>
#@markdown Tus tags deben ser relevantes para lo que desees entrenar, y excluir elementos no deseados (el contenido explícito puede hacer más difícil el entrenamiento).
#@markdown Las palabras van separadas por guionbajos, las tags van separadas por espacios, y usa - para excluir esa tag. También puedes incluir una puntuación mínima: `score:>10`
tags = "1girl -sex -greyscale -monochrome" #@param {type:"string"}
#markdown Si una imagen supera esta resolución, se descargará una versión más pequeña.
max_resolution = 3072 #param {type:"slider", min:1024, max:8196, step:1024}
#markdown Posts with a parent post are often minor variations of the same image.
include_posts_with_parent = True #param {type:"boolean"}

tags = tags.replace(" ", "+")\
           .replace("(", "%28")\
           .replace(")", "%29")\
           .replace(":", "%3a")\

url = "https://gelbooru.com/index.php?page=dapi&json=1&s=post&q=index&limit=100&tags={}".format(tags)
user_agent = "Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Gecko; compatible; Googlebot/2.1; +http://www.google.com/bot.html) Chrome/93.0.4577.83 Safari/537.36"
limit = 100 # hardcoded by gelbooru
total_limit = 1000 # you can edit this if you want but I wouldn't recommend it
supported_types = (".png", ".jpg", ".jpeg")

def ubuntu_deps():
  print("🏭 Instalando...\n")
  !apt update
  !apt -y install aria2
  return not get_ipython().__dict__['user_ns']['_exit_code']

if "step2_installed_flag" not in globals():
  if ubuntu_deps():
    clear_output()
    step2_installed_flag = True
  else:
    print("❌ Error en la instalación, intentando continuar...")

def get_json(url):
  with urlopen(Request(url, headers={"User-Agent": user_agent})) as page:
    return json.load(page)

def filter_images(data):
  return [p["file_url"] if p["width"]*p["height"] <= max_resolution**2 else p["sample_url"]
          for p in data["post"]
          if (p["parent_id"] == 0 or include_posts_with_parent)
          and p["file_url"].lower().endswith(supported_types)]

def download_images():
  data = get_json(url)
  count = data["@attributes"]["count"]

  if count == 0:
    print("📷 No se encontraron resultados.")
    return

  print(f"🎯 Se encontraron {count} resultados")
  test_url = "https://gelbooru.com/index.php?page=post&s=list&tags={}".format(tags)
  display(Markdown(f"[¡Click aquí para verlos en tu navegador!]({test_url})"))
  print (f"🔽 Se descargará en {images_folder.replace('/content/drive/', '')} (Aparecerá una confirmación aquí abajo, sino vuelve a correr esta celda)")
  inp = input("❓ Escribe \"si\" para continuar con la descarga: ")

  if inp.lower().strip() not in ("si", "sí"):
    print("❌ Download cancelled")
    return

  print("📩 Obteniendo lista de imágenes...")

  image_urls = set()
  image_urls = image_urls.union(filter_images(data))
  for i in range(total_limit // limit):
    count -= limit
    if count <= 0:
      break
    time.sleep(0.1)
    image_urls = image_urls.union(filter_images(get_json(url+f"&pid={i+1}")))

  scrape_file = os.path.join(config_folder, f"scrape_{project_subfolder}.txt")
  with open(scrape_file, "w") as f:
    f.write("\n".join(image_urls))

  print(f"🌐 Enlaces guardados a {scrape_file}\n\n🔁 Descargando imágenes ...\n")
  old_img_count = len([f for f in os.listdir(images_folder) if f.lower().endswith(supported_types)])

  os.chdir(images_folder)
  !aria2c --console-log-level=warn -c -x 16 -k 1M -s 16 -i {scrape_file}

  new_img_count = len([f for f in os.listdir(images_folder) if f.lower().endswith(supported_types)])
  print(f"\n✅ Se han descargado {new_img_count - old_img_count} imágenes.")

download_images()


In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

#@markdown ### 3️⃣ Filtrar tus imágenes
#@markdown Vamos a detectar y borrar imágenes duplicadas usando la IA de FiftyOne. <p>
#@markdown Porcentaje de similaridad para que dos imágenes se consideren iguales. Recomiendo 0.97 to 0.99:
similaridad = 0.985 #@param {type:"number"}
similarity_threshold = similaridad
#@markdown Puedes elegir entre simplemente borrar los duplicados, o adicionalmente abrir un área interactiva bajo esta celda para visualizar tus imágenes y marcar las que quieras con `delete` que luego serán borradas. <p>
#@markdown Si el área interactiva no aparece o está vacía, intenta activar las cookies y quitar la protección contra rastreo para la página de Google Colab en tu navegador.
#@markdown Para guardar los cambios del área interactiva tendrás que apretar Enter en la caja de texto que aparecerá sobre el area interactiva.<p>
accion = "Borrar duplicados" #@param ["Borrar duplicados","Marcar duplicados y abrir area interactiva","Abrir area interactiva"]
action = accion

open_in_new_tab = False
ngrok_token = ""


os.chdir(root_dir)
model_name = "clip-vit-base32-torch"
supported_types = (".png", ".jpg", ".jpeg")
img_count = len(os.listdir(images_folder))
batch_size = min(250, img_count)

if "step3_installed_flag" not in globals():
  print("🏭 Instalando dependencias...\n")
  !pip -q install fiftyone ftfy pyngrok
  !pip -q install fiftyone-db-ubuntu2204
  if not get_ipython().__dict__['user_ns']['_exit_code']:
    clear_output()
    step3_installed_flag = True
  else:
    print("❌ Error de instalación, intentando continuar de todas formas...")

os.environ["FIFTYONE_SERVER"] = "0"
import numpy as np
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
from sklearn.metrics.pairwise import cosine_similarity
from pyngrok import ngrok, conf
from portpicker import pick_unused_port

non_images = [f for f in os.listdir(images_folder) if not f.lower().endswith(supported_types)]
if non_images:
  print(f"💥 Error: El archivo {non_images[0]} no es una imagen - Esto no está permitido. Usa los Extras más abajo para borrar los archivos que no sean imágenes.")
elif img_count == 0:
  print(f"💥 Error: No se encontraron imágenes en {images_folder}")
else:
  print("\n💿 Analizando dataset...\n")
  dataset = fo.Dataset.from_dir(images_folder, dataset_type=fo.types.ImageDirectory)
  if "duplicados" in action:
    model = foz.load_zoo_model(model_name)
    embeddings = dataset.compute_embeddings(model, batch_size=batch_size)

    batch_embeddings = np.array_split(embeddings, batch_size)
    similarity_matrices = []
    max_size_x = max(array.shape[0] for array in batch_embeddings)
    max_size_y = max(array.shape[1] for array in batch_embeddings)

    for i, batch_embedding in enumerate(batch_embeddings):
      similarity = cosine_similarity(batch_embedding)
      #Pad 0 for np.concatenate
      padded_array = np.zeros((max_size_x, max_size_y))
      padded_array[0:similarity.shape[0], 0:similarity.shape[1]] = similarity
      similarity_matrices.append(padded_array)

    similarity_matrix = np.concatenate(similarity_matrices, axis=0)
    similarity_matrix = similarity_matrix[0:embeddings.shape[0], 0:embeddings.shape[0]]

    similarity_matrix = cosine_similarity(embeddings)
    similarity_matrix -= np.identity(len(similarity_matrix))

    dataset.match(F("max_similarity") > similarity_threshold)
    dataset.tags = ["delete", "has_duplicates"]

    id_map = [s.id for s in dataset.select_fields(["id"])]
    samples_to_remove = set()
    samples_to_keep = set()

    for idx, sample in enumerate(dataset):
      if sample.id not in samples_to_remove:
        # Keep the first instance of two duplicates
        samples_to_keep.add(sample.id)

        dup_idxs = np.where(similarity_matrix[idx] > similarity_threshold)[0]
        for dup in dup_idxs:
            # We kept the first instance so remove all other duplicates
            samples_to_remove.add(id_map[dup])

        if len(dup_idxs) > 0:
            sample.tags.append("has_duplicates")
            sample.save()
      else:
        sample.tags.append("delete")
        sample.save()

    sidebar_groups = fo.DatasetAppConfig.default_sidebar_groups(dataset)
    for group in sidebar_groups[1:]:
      group.expanded = False
    dataset.app_config.sidebar_groups = sidebar_groups
    dataset.save()

  if "interactiva" in action:
    clear_output()
    os.environ["FIFTYONE_SERVER"] = "1"
    port = pick_unused_port()
    session = fo.launch_app(dataset, port=port, auto=not open_in_new_tab)
    if open_in_new_tab:
      conf.get_default().auth_token = ngrok_token
      public_url = ngrok.connect(port).public_url
      print(f"🟢 Sesión abierrta en {public_url}")

    print("❗ Espera un minuto para que cargue la sesión, sino, revisa las instrucciones arriba.")
    print("❗ Cuando esté listo, verás una cuadrícula con tus imágenes.")
    print("❗ A la izquierda puedes activar \"sample tags\" para ver las imágenes que están marcadas.")
    print("❗ Puedes marcar las imágenes que quieras tras haberlas seleccionado, poniendo un tag llamado \"delete\" en la parte superior.")
    input("⭕ Cuando estés listo, debes guardar los cambios enviando Enter en esta caja de texto: ")

    print("💾 Saving...")

  marked = [s for s in dataset if "delete" in s.tags]
  dataset.delete_samples(marked)
  previous_folder = images_folder[:images_folder.rfind("/")]
  dataset.export(export_dir=os.path.join(images_folder, project_subfolder), dataset_type=fo.types.ImageDirectory)

  temp_suffix = "_temp"
  !mv {images_folder} {images_folder}{temp_suffix}
  !mv {images_folder}{temp_suffix}/{project_subfolder} {images_folder}
  !rm -r {images_folder}{temp_suffix}

  if "interactiva" in action:
    session.refresh()
    fo.close_app()
    clear_output()

  print(f"\n✅ {len(marked)} imágenes han sido borradas. Ahora tienes {len(os.listdir(images_folder))} imágenes.")


In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Por favor corre el paso 1 primero!")

#@markdown ### 4️⃣ Generar descripciones
#@markdown Usaremos inteligencia artificial para describir tus imágenes, específicamente [Waifu Diffusion](https://huggingface.co/SmilingWolf/wd-eva02-large-tagger-v3) en el caso de anime (etiquetas/tags) y [BLIP](https://huggingface.co/spaces/Salesforce/BLIP) en el caso de fotografías (subtítulos/captions) <p>
#@markdown Estas descripciones que van junto a tus imágenes mejoran notablemente la calidad de tu Lora a la hora de entrenar. El proceso demora 5 minutes en instalar y 5 minutos más para describir 1000 imágenes. Recorre todas las subcarpetas si existen. <p>
metodo = "Tags de Anime" #@param ["Tags de Anime", "Descripciones de Fotos"]
method = metodo
#@markdown **Anime:** Usar ambos taggers es más preciso que uno o el otro. Menor umbral generará más tags, prueba 0.25 para conceptos y 0.50 para estilos. Deberías incluir nombres de personajes si no estás entrenando un personaje.
tagger = "Ambos" #@param ["Ambos","SmilingWolf/wd-eva02-large-tagger-v3","SmilingWolf/wd-vit-large-tagger-v3"]
umbral = 0.25 #@param {type:"slider", min:0.0, max:1.0, step:0.01}
tag_threshold = umbral
tags_no_permitidas = "virtual youtuber, parody, style parody, official alternate costume, official alternate hairstyle, official alternate hair length, alternate costume, alternate hairstyle, alternate hair length, alternate hair color" #@param {type:"string"}
blacklist_tags = tags_no_permitidas
nombres_de_personajes = False #@param {type:"boolean"}
include_character_names = nombres_de_personajes
#@markdown **Fotografías:** El mínmimo y máximo largo de cada subtítulo (medido en tokens/palabras).
largo_minimo = 10 #@param {type:"number"}
caption_min = largo_minimo
largo_maximo = 75 #@param {type:"number"}
caption_max = largo_maximo

character_threshold = tag_threshold if include_character_names else 1.1
undesired_tags = '"' + ','.join([t.strip() for t in blacklist_tags.split(",") if t.strip()]) + '"'

kohya_dir = "/content/kohya"
venv_python = os.path.join(kohya_dir, "venv/bin/python")
venv_pip = os.path.join(kohya_dir, "venv/bin/pip")

if "step4_installed_flag" not in globals():
  print("\n🏭 Instalando dependencias...\n")
  !apt update
  !apt install -y python3.10-venv -qq
  !git clone https://github.com/kohya-ss/sd-scripts {kohya_dir}
  os.chdir(kohya_dir)
  !git reset --hard e89653975ddf429cdf0c0fd268da0a5a3e8dba1f
  !python3.10 -m venv venv
  !{venv_pip} install -r requirements.txt
  !{venv_pip} install fairscale==0.4.13 timm==0.6.12
  !{venv_pip} install onnx onnxruntime-gpu==1.20.1 --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
  !{venv_pip} uninstall -y rich
  step4_installed_flag = True

print("\n🚶‍♂️ Corriendo programa...\n")
os.chdir(kohya_dir)

if "Anime" in method:
  tagger_models = [
    "SmilingWolf/wd-eva02-large-tagger-v3",
    "SmilingWolf/wd-vit-large-tagger-v3"
  ] if tagger == "Ambos" else [tagger]

  for i, tagger_model in enumerate(tagger_models):
    append_tags = "--append_tags" if i > 0 else ""
    !{venv_python} finetune/tag_images_by_wd14_tagger.py \
      {images_folder} \
      --repo_id={tagger_model} \
      --general_threshold={tag_threshold} \
      --character_threshold={character_threshold} \
      --batch_size=8 \
      --max_data_loader_n_workers=2 \
      --caption_extension=.txt \
      --undesired_tags {undesired_tags} \
      --onnx --recursive --remove_underscore {append_tags}

  if not get_ipython().__dict__['user_ns']['_exit_code']:
    # Count tags
    from collections import Counter
    text_files = []
    for root, dirs, files in os.walk(images_folder):
      for file in files:
        if file.lower().endswith(".txt"):
          text_files.append(os.path.join(root, file))
    top_tags = Counter()
    for file in text_files:
      with open(file, 'r') as f:
        tags = [t.strip() for t in f.read().split(",")]
      top_tags.update(tags)

    clear_output()
    print(f"📊 Tagging finalizado. Aquí están las 50 imágenes más comunes en tu dataset:")
    print("\n".join(f"{k} ({v})" for k, v in top_tags.most_common(50)))
  
else:
  !{venv_python} finetune/make_captions.py \
    {images_folder} \
    --beam_search \
    --max_data_loader_n_workers=2 \
    --batch_size=8 \
    --min_length={caption_min} \
    --max_length={caption_max} \
    --caption_extension=.txt \
    --recursive

  if not get_ipython().__dict__['user_ns']['_exit_code']:
    import random
    for root, dirs, files in os.walk(images_folder):
      for file in files:
        if file.lower().endswith(".txt"):
          text_files.append(os.path.join(root, file))
    sample = []
    for txt in random.sample(captions, min(10, len(captions))):
      with open(os.path.join(images_folder, txt), 'r') as f:
        sample.append(f.read())

    clear_output()
    print(f"📊 Captioning finalizado. Aquí hay {len(sample)} ejemplos de descripciones en tu dataset:")
    print("".join(sample))

os.chdir(root_dir)

In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Por favor usa el paso 1 primero.")

#@markdown ### 5️⃣ Editar las tags
#@markdown Paso opcional para tags de anime. Puedes correr este paso cuantas veces quieras. <p>

#@markdown Añadir una palabra de activación a tu Lora, útil para mejorar el entrenamiento y usarlo en tus prompts. En el entrenamiento debes poner `keep_tokens` igual a 1.<p>
#@markdown Si quitas tags comunes como el color de pelo/ojos éstas serán "absorbidas" por tu palabra de activación.
palabra_de_activacion = "" #@param {type:"string"}
global_activation_tag = palabra_de_activacion
quitar_tags = "" #@param {type:"string"}
remove_tags = quitar_tags
#@markdown <p>&nbsp;<p> En esta zona avanzada puedes realizar reemplazos o combinaciones de tags para así mejorar su calidad. Puedes reemplazar 1 tag por varias, o varias por 1, o una por otra, etc. También puedes añadir palabras de activación específicas.
buscar_tags = "" #@param {type:"string"}
search_tags = buscar_tags
reemplazar_con = "" #@param {type:"string"}
replace_with = reemplazar_con
modo_de_busqueda = "OR (puede tener cualquiera)" #@param ["OR (puede tener cualquiera)", "AND (debe tener todo)"]
search_mode = modo_de_busqueda
tag_nueva_se_convierte_en_palabra_de_activacion = False #@param {type:"boolean"}
new_becomes_activation_tag = tag_nueva_se_convierte_en_palabra_de_activacion
#@markdown Estas pueden ser útiles a veces. Ten cuidado, pueden quitar las palabras de activación previas.
ordenar_alfabeticamente = False #@param {type:"boolean"}
sort_alphabetically = ordenar_alfabeticamente
quitar_duplicados = False #@param {type:"boolean"}
remove_duplicates = quitar_duplicados

def split_tags(tagstr):
  return [s.strip() for s in tagstr.split(",") if s.strip()]

activation_tag_list = split_tags(global_activation_tag)
remove_tags_list = split_tags(remove_tags)
search_tags_list = split_tags(search_tags)
replace_with_list = split_tags(replace_with)
replace_new_list = [t for t in replace_with_list if t not in search_tags_list]

replace_with_list = [t for t in replace_with_list if t not in replace_new_list]
replace_new_list.reverse()
activation_tag_list.reverse()

remove_count = 0
replace_count = 0

text_files = []
for root, dirs, files in os.walk(images_folder):
  for file in files:
    if file.lower().endswith(".txt"):
      text_files.append(os.path.join(root, file))

for txt in text_files:

  with open(os.path.join(images_folder, txt), 'r') as f:
    tags = [s.strip() for s in f.read().split(",")]

  if remove_duplicates:
    tags = list(set(tags))
  if sort_alphabetically:
    tags.sort()

  for rem in remove_tags_list:
    if rem in tags:
      remove_count += 1
      tags.remove(rem)

  if "AND" in search_mode and all(r in tags for r in search_tags_list) \
      or "OR" in search_mode and any(r in tags for r in search_tags_list):
    replace_count += 1
    for rem in search_tags_list:
      if rem in tags:
        tags.remove(rem)
    for add in replace_with_list:
      if add not in tags:
        tags.append(add)
    for new in replace_new_list:
      if new_becomes_activation_tag:
        if new in tags:
          tags.remove(new)
        tags.insert(0, new)
      else:
        if new not in tags:
          tags.append(new)

  for act in activation_tag_list:
    if act in tags:
      tags.remove(act)
    tags.insert(0, act)

  with open(os.path.join(images_folder, txt), 'w') as f:
    f.write(", ".join(tags))

if remove_tags:
  print(f"\n🚮 Se han quitado {remove_count} tags.")
if search_tags:
  print(f"\n💫 Se han hecho {replace_count} reemplazos.")
print("\n✅ ¡Listo!")


In [ ]:
#@markdown ### 6️⃣  Listo
#@markdown Ahora debes estar listo para [entrenar tu Lora](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb).

from IPython.display import Markdown, display
display(Markdown(f"🦀 [Click aquí para abrir el colab de entrenamiento](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) "))


## *️⃣ Extras

In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Por favor usa el paso 1 primero.")

#@markdown ### 📈 Analizar tags
#@markdown Volver a ver las tags más comunes en tus imágenes.
ver_top = 50 #@param {type:"number"}
show_top_tags = ver_top

text_files = []
for root, dirs, files in os.walk(images_folder):
  for file in files:
    if file.lower().endswith(".txt"):
      text_files.append(os.path.join(root, file))

from collections import Counter
top_tags = Counter()
for file in text_files:
  with open(file, 'r') as f:
    tags = [t.strip() for t in f.read().split(",")]
  top_tags.update(tags)

print(f"📊 Tus {show_top_tags} tags más comunes:")
for k, v in top_tags.most_common(show_top_tags):
  print(f"{k} ({v})")

In [ ]:
#@markdown ### 📂 Extraer datos
#@markdown Es lento subir muchos archivos pequeños, si quieres puedes subir un zip y extraerlo aquí.
zip = "/content/drive/MyDrive/Loras/warrior.zip" #@param {type:"string"}
extract_to = "/content/drive/MyDrive/Loras/ejemplo/dataset" #@param {type:"string"}

import os, zipfile

if not os.path.exists('/content/drive'):
  from google.colab import drive
  print("📂 Connecting to Google Drive...")
  drive.mount('/content/drive')

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip, 'r') as f:
  f.extractall(extract_to)

print("✅ Done")


In [ ]:
#@markdown ### 🔢 Contar archivos
#@markdown Google Drive hace imposible contar los archivos en una carpeta, por lo que aquí puedes ver la cantidad de archivos en carpetas y subcarpetas.
carpeta = "/content/drive/MyDrive/Loras" #@param {type:"string"}
folder = carpeta

import os
from google.colab import drive

if not os.path.exists('/content/drive'):
    print("📂 Connecting to Google Drive...\n")
    drive.mount('/content/drive')

tree = {}
exclude = ("_logs", "/output")
for i, (root, dirs, files) in enumerate(os.walk(folder, topdown=True)):
  dirs[:] = [d for d in dirs if all(ex not in d for ex in exclude)]
  images = len([f for f in files if f.lower().endswith((".png", ".jpg", ".jpeg"))])
  captions = len([f for f in files if f.lower().endswith(".txt")])
  others = len(files) - images - captions
  path = root[folder.rfind("/")+1:]
  tree[path] = None if not images else f"{images:>4} images | {captions:>4} captions |"
  if tree[path] and others:
    tree[path] += f" {others:>4} other files"

pad = max(len(k) for k in tree)
print("\n".join(f"📁{k.ljust(pad)} | {v}" for k, v in tree.items() if v))


In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

from PIL import Image
import os
Image.MAX_IMAGE_PIXELS = None

#@markdown ### 🖼️ Reducir tamaño de imágenes
#@markdown Esto convertirá todas las imágenes en la carpeta del proyecto a jpeg, reduciendo el tamaño sin afectar mucho la calidad. Esto también puede solucionar algunos errores.
location = images_folder

for dir in [d[0] for d in os.walk(location)]:
    os.chdir(dir)
    converted = False
    for file_name in list(os.listdir(".")):
        try:
            # Convert png to jpeg
            if file_name.endswith(".png"):
                if not converted:
                    print(f"Converting {dir}")
                    converted = True
                im = Image.open(file_name)
                im = im.convert("RGB")
                new_file_name = os.path.splitext(file_name)[0] + ".jpeg"
                im.save(new_file_name, quality=95)
                os.remove(file_name)
                file_name = new_file_name
            # Resize large jpegs
            if file_name.endswith((".jpeg", ".jpg")) and os.path.getsize(file_name) > 2000000:
                if not converted:
                    print(f"Converting {dir}")
                    converted = True
                im = Image.open(file_name)
                im = im.resize((int(im.width/2), int(im.height/2)))
                im.save(file_name, quality=95)
            # Rename jpg to jpeg
            if file_name.endswith(".jpg"):
                if not converted:
                    print(f"Converting {dir}")
                new_file_name = os.path.splitext(file_name)[0] + ".jpeg"
                os.rename(file_name, new_file_name)
        except Exception as e:
            print(f"An error occurred while processing {file_name}: {e}")
    if converted:
        print(f"Converted {dir}")


In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Por favor usa el paso 1 primero.")

#@markdown ### 🚮 Limpiar carpeta
#@markdown Cuidado, borra todos los archivos que no sean imágenes de la carpeta del proyecto.

!find {images_folder} -type f ! \( -name '*.png' -o -name '*.jpg' -o -name '*.jpeg' \) -delete